# test

In [ ]:
# base setting
import sys
sys.path.append('../src/')

In [6]:
# libraries
from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
import pandas as pd
from sklearn.linear_model import LogisticRegression
from process import texthandling as th

In [7]:
# logger setting
logger = getLogger(__name__)
logdir = 'log/'
if __name__ == '__main__':
    log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s][%(funcName)s] %(message)s')
    handler = StreamHandler()
    handler.setLevel('INFO')
    handler.setFormatter(log_fmt)
    logger.addHandler(handler)
    
    handler = FileHandler(logdir + 'logistic_reg.ipynb.log', 'a')
    handler.setLevel(DEBUG)
    handler.setFormatter(log_fmt)
    logger.setLevel(DEBUG)
    logger.addHandler(handler)

In [16]:
# data input
datadir = '../input/'
train = pd.read_csv(datadir + 'train.csv')

In [19]:
# processing

# get dummy variables
dum_sex = pd.get_dummies(train['Sex'])
dum_emb = pd.get_dummies(train['Embarked'])

train_proc = pd.concat((train, dum_sex, dum_emb), axis = 1)
train_proc = train_proc.drop('Sex', axis = 1)
train_proc = train_proc.drop('Embarked', axis = 1)
train_proc = train_proc.drop('female', axis = 1)
train_proc = train_proc.drop('S', axis = 1)

name_df = th.FlagWords(train_proc['Name'], ['Mr', 'Master', 'Mrs', 'Miss', ])
train_proc = pd.concat([train_proc, name_df], axis=1)
train_proc = train_proc.drop('Name', axis=1)

train_proc.head(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,male,C,Q,Mr,Master,Mrs,Miss
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,1,0,0,1,0,0,0
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,0,1,0,1,0,1,0
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,0,0,0,0,1
3,4,1,1,35.0,1,0,113803,53.1000,C123,0,0,0,1,0,1,0
4,5,0,3,35.0,0,0,373450,8.0500,NaN,1,0,0,1,0,0,0
